In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

### 抓取所有Html信息


In [34]:
driver = webdriver.Chrome()

url='https://lf.esf.fang.com/house-a010278/'
driver.get(url)

html_list = []
i = 0
Num_Pages = 20


while i < Num_Pages:
    try:
        # 获取当前页所有房源的 dl 列表
        house_items = driver.find_elements(By.CSS_SELECTOR, ".shop_list dl")
        for item in house_items:
            html_list.append(item.get_attribute('outerHTML'))
        
        print(f"第 {i+1} 页抓取完成，共 {len(house_items)} 套房源")

        # 查找“下一页”按钮
        next_page = driver.find_element('class name', 'last')
        next_page.click()
        i += 1
    except NoSuchElementException:
        break


第 1 页抓取完成，共 60 套房源
第 2 页抓取完成，共 60 套房源
第 3 页抓取完成，共 60 套房源
第 4 页抓取完成，共 60 套房源
第 5 页抓取完成，共 60 套房源
第 6 页抓取完成，共 60 套房源
第 7 页抓取完成，共 60 套房源
第 8 页抓取完成，共 60 套房源
第 9 页抓取完成，共 60 套房源
第 10 页抓取完成，共 60 套房源
第 11 页抓取完成，共 60 套房源
第 12 页抓取完成，共 60 套房源
第 13 页抓取完成，共 60 套房源
第 14 页抓取完成，共 60 套房源
第 15 页抓取完成，共 60 套房源
第 16 页抓取完成，共 60 套房源
第 17 页抓取完成，共 60 套房源
第 18 页抓取完成，共 60 套房源
第 19 页抓取完成，共 60 套房源
第 20 页抓取完成，共 60 套房源


### 解析HTML列表



In [35]:
all_properties = []

for html in html_list:
    soup = BeautifulSoup(html, 'html.parser')
    property_info = {}

    # 总价 
    price_b = soup.find('span', class_='red').find('b') if soup.find('span', class_='red') else None
    price_unit = soup.find('span', class_='red').get_text(strip=True) if soup.find('span', class_='red') else ""

    if price_b:
        total_price_text = price_b.get_text(strip=True) + '万'  # 如 "242万"
    elif '万' in price_unit:
        match = re.search(r'(\d+\.?\d*)万', price_unit)  # 支持小数如 "242.5万"
        total_price_text = match.group(1) + '万' if match else None
    else:
        total_price_text = None

    if total_price_text and total_price_text != "未知":
        try:
            # 提取数字部分（去掉“万”字）
            num_str = total_price_text.replace('万', '')
            total_price = int(float(num_str) * 10000)
        except ValueError:
            total_price = None
    else:
        total_price = None  # 或者用 "未知"，但推荐 None 更利于后续处理

    property_info['总价'] = total_price  # 已是整数，如 2420000

    # 单价 
    unit_price_span = soup.find('dd', class_='price_right').find('span', string=re.compile(r'元/㎡'))
    
    if unit_price_span:
        raw_text = unit_price_span.get_text(strip=True)
        # 提取数字
        num_match = re.search(r'(\d+\.?\d*)', raw_text)
        property_info['单价'] = float(num_match.group(1)) if num_match else None
    else:
        property_info['单价'] = None  # 表示缺失


    # --- 解析 <p class="tel_shop"> 中的详细信息 ---
    tel_shop = soup.find('p', class_='tel_shop')
    if tel_shop:
        # 移除 <a> 和 <span> 标签
        for tag in tel_shop.find_all(['a', 'span']):
            tag.extract()

        text = tel_shop.get_text(strip=True)
        parts = [part.strip() for part in text.split('|')]

        # 从 parts 中提取面积：通常第一个是 "xx㎡"
        area = "未知"
        for part in parts:
            if '㎡' in part:
                area_match = re.search(r'(\d+\.?\d*)㎡', part)
                if area_match:
                    area = float(area_match.group(1))
                break
        property_info['面积'] = area
    else:
        property_info['面积'] = "未知"

    all_properties.append(property_info)

# 转为 DataFrame
df = pd.DataFrame(all_properties)


### 保存到 Excel 和 CSV

In [36]:
if not df.empty:
    df.to_csv("DACHANG_Housing_Price.csv", index=False, encoding='utf-8-sig')
    print(f"成功抓取 {len(df)} 条房源数据")
else:
    print("未能抓取到任何数据")

# 显示前几行
print(df.head())

driver.quit()

成功抓取 1200 条房源数据
        总价      单价      面积
0   790000  8977.0   88.00
1   780000  6702.0  116.38
2  1600000  6451.0  248.00
3  1500000  6818.0  220.00
4   670000  7528.0   89.00


## 爬取租房价格


In [37]:
driver = webdriver.Chrome()

url='https://lf.zu.fang.com/house-a010278/n31/' #只保留整租
driver.get(url)

In [38]:
html_list_rent = []
i = 0
Num_Pages = 20
while i < Num_Pages:
    try:
        # 获取当前页所有房源的 dl 列表
        rent_items = driver.find_elements(By.CSS_SELECTOR, ".houseList dl")
        for item in rent_items:
            html_list_rent.append(item.get_attribute('outerHTML'))
        
        print(f"第 {i+1} 页抓取完成，共 {len(rent_items)} 套房源")

        # 查找“下一页”按钮
        next_page = driver.find_element(By.LINK_TEXT, '下一页')
        next_page.click()
        i += 1
    except NoSuchElementException:
        break

第 1 页抓取完成，共 60 套房源
第 2 页抓取完成，共 60 套房源
第 3 页抓取完成，共 60 套房源
第 4 页抓取完成，共 60 套房源
第 5 页抓取完成，共 60 套房源
第 6 页抓取完成，共 60 套房源
第 7 页抓取完成，共 60 套房源
第 8 页抓取完成，共 60 套房源
第 9 页抓取完成，共 60 套房源
第 10 页抓取完成，共 60 套房源
第 11 页抓取完成，共 60 套房源
第 12 页抓取完成，共 60 套房源
第 13 页抓取完成，共 60 套房源
第 14 页抓取完成，共 60 套房源
第 15 页抓取完成，共 60 套房源
第 16 页抓取完成，共 60 套房源
第 17 页抓取完成，共 60 套房源
第 18 页抓取完成，共 60 套房源
第 19 页抓取完成，共 60 套房源
第 20 页抓取完成，共 60 套房源


### 解析 HTML 列表

In [39]:
all_rent_properties = []

for html in html_list_rent:
    soup = BeautifulSoup(html, 'html.parser')
    property_info = {}

 # --- 1. 月租（纯数字）---
    price_span = soup.find('span', class_='price')
    if price_span:
        raw_price = price_span.get_text(strip=True)
        # 提取数字部分
        num_match = re.search(r'(\d+)', raw_price)
        property_info['月租'] = int(num_match.group(1)) if num_match else None
    else:
        property_info['月租'] = None

    # --- 2. 面积 ---
    # 在 class="font15 mt12 bold" 的 p 标签中
    info_p = soup.find('p', class_='font15 mt12 bold')
    if info_p:
        text = info_p.get_text(strip=True)
        # 使用 '|' 分割，去除空格
        parts = [part.strip() for part in text.split('|') if part.strip()]

        # 面积：找包含 '㎡' 的部分
        area_part = None
        for part in parts:
            if '㎡' in part:
                area_part = part
                break

        if area_part:
            num_match = re.search(r'(\d+\.?\d*)', area_part)  # 支持小数
            property_info['面积'] = float(num_match.group(1)) if num_match else None
        else:
            property_info['面积'] = None
       
    # --- 4. 计算每平米月租 ---
    if property_info['月租'] and property_info['面积']:
        property_info['每平米月租'] = round(property_info['月租'] / property_info['面积'], 2)
    else:
        property_info['每平米月租'] = None

    all_rent_properties.append(property_info)

# 转为 DataFrame（需要 pandas）
import pandas as pd
df_rent = pd.DataFrame(all_rent_properties)

print(df_rent.head().to_string(index=False))


  月租    面积  每平米月租
6200 250.0  24.80
 700  78.0   8.97
 600  73.0   8.22
1400  79.0  17.72
1000  89.0  11.24


In [40]:
if not df_rent.empty:
    df_rent.to_csv("DACHANG_Rent_Price.csv", index=False, encoding='utf-8-sig')
    print(f"成功抓取 {len(df_rent)} 条房源数据")
else:
    print("未能抓取到任何数据")

# 显示前几行
print(df_rent.head())

driver.quit()

成功抓取 1200 条房源数据
     月租     面积  每平米月租
0  6200  250.0  24.80
1   700   78.0   8.97
2   600   73.0   8.22
3  1400   79.0  17.72
4  1000   89.0  11.24
